# deeplodocus-learn

#### За основу было взято: [Learning to Play Chess with Minimal Lookahead and Deep Value Neural Networks](https://www.ai.rug.nl/~mwiering/Thesis_Matthia_Sabatelli.pdf)

#### Мои обученные модели: [Google drive](https://drive.google.com/drive/folders/1VvwrBkgA9i_8i3D2PHZ_CJ_ni2xFUPlT)

<p style="text-align:center;"><img src="media/logo.png" alt="drawing" width="200"/></p>

In [1]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, BatchNormalization
import keras
import pandas as pd
import stockfish
import chess, chess.pgn
import io
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics
import numpy as np

# Подгружаем наши датасеты
X = np.load('final/Positions.npy')
y = np.load('final/Labels.npy')

2023-09-06 21:03:58.134412: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
model = Sequential()

# Вход интерпретируется как одномерный массив размера 12х8х8=768
# Был использован трехслойный MLP с 2048 скрытыми единицами на слой
# Каждый слой активируется функцией elu
# После каждого скрытого слоя используется BatchNormalization
# Для задачи регрессии используется выход равный 1 с функцией активации linear

model.add(Dense(2048, activation='elu', input_shape=(768,)))
model.add(BatchNormalization())

model.add(Dense(2048, activation='elu'))
model.add(BatchNormalization())

model.add(Dense(2048, activation='elu'))
model.add(BatchNormalization())

model.add(Dense(1, activation='linear'))

model.summary()

sgd = keras.optimizers.legacy.SGD(learning_rate=0.001, decay=1e-8, momentum=0.7, nesterov=True)
model.compile(loss='mean_squared_error', optimizer=sgd)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 2048)              1574912   
                                                                 
 batch_normalization (Batch  (None, 2048)              8192      
 Normalization)                                                  
                                                                 
 dense_1 (Dense)             (None, 2048)              4196352   
                                                                 
 batch_normalization_1 (Bat  (None, 2048)              8192      
 chNormalization)                                                
                                                                 
 dense_2 (Dense)             (None, 2048)              4196352   
                                                                 
 batch_normalization_2 (Bat  (None, 2048)              8

In [ ]:
# Обучаем модель
history = model.fit(X, y, batch_size=248, epochs=500)

# Сохраняем модель в файл
model.save('1_400_000_ep500.h5')